# Charis Project 4 Part 2: Latent Dirichlet Allocation
LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each job description is modeled as a multinomial distribution of job title and each job title is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution.

In [37]:
import json
import pandas as pd

## Step 1: Load the dataset

The dataset we'll use is a list of job descriptions, start by loading the text file.

In [38]:
# Load the dataset from previous section (in a separate jupyter notebook) and save it to 'data_text'
# with open('data1.txt') as json_file:  
#     data = json.load(json_file)

# documents = pd.DataFrame.from_dict(data).T.description
# nrows = documents.shape[0]

In [39]:
documents = pd.read_csv('cleanIndeedMelbourneMay20.csv').description
nrows = documents.shape[0]

In [40]:
documents.head()

0    The Opportunity\nImmerse Yourself in an Inclus...
1    Company:\nThe company is a cutting edge crypto...
2    Job DescriptionBackend Engineer (Integration) ...
3    Want to work on the latest tech/ML and join a ...
4    Job Description\nSoftware Engineer – Cognitive...
Name: description, dtype: object

Let's glance at the dataset:

In [41]:
# Get the total number of documents
print(len(documents))

76


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [42]:
# Loading Gensim and nltk libraries
# import sys
# !{sys.executable} -m pip install nltk numpy tqdm gensim

In [43]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
import numpy as np
np.random.seed(400)

In [44]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [45]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return (WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            # Apply lemmatize_stemming on the token, then add to the results list
            result.append(lemmatize_stemming(token))
    return result

In [46]:
# Preview a document after preprocessing
document_num = 3  #taking the document at index 3 as sample for demonstration
doc_sample = documents[document_num]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['Want', 'to', 'work', 'on', 'the', 'latest', 'tech/ML', 'and', 'join', 'a', 'company', 'at', 'the', 'beginning', 'of', 'their', 'success', 'journey?\nStart-Up', 'that', 'has', 'gone', 'international\nNear', 'Parliament', 'station\n$60K', '-', '$100K', '+', 'Super\nCompany\nA', 'young', 'and', 'nimble', 'start-up', 'has', 'gone', 'international,', 'so', 'it', 'is', 'only', 'natural', 'to', 'open', 'our', 'doors', 'to', 'more', 'talented', 'software', 'engineers.\nHere,', 'we', 'build', 'cutting', 'edge', 'productivity/project', 'management', 'software', 'for', 'the', 'construction', 'industry', 'and', 'we', 'are', 'doing', 'some', 'pretty', 'cool', 'things!', 'These', 'tools', 'nobody', 'has', 'built', 'before', 'leading', 'to', 'huge', 'success', 'and', 'our', 'next', 'phase', 'will', 'be', 'incorporating', 'Machine', 'Learning.\n\nRole\nDay', 'to', 'day', 'you', 'will', 'be', 'working', 'on', 'our', 'core', 'JavaScript', 'application.', 'We', 'are', 'currently', '

Let's now preprocess all the job descriptions we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `title` column

In [47]:
# preprocess all the job descriptions, saving the list of results as 'processed_docs'
processed_docs = documents.map(preprocess)

TypeError: decoding to str: need a bytes-like object, float found

In [ ]:
#Preview 'processed_docs', remember processed_docs is a pandas series
processed_docs[:10]

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [ ]:
#Create a dictionary from 'processed_docs' containing the number of times a word appears in the training set using gensim.corpora.Dictionary and call it 'dictionary'
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
#for example, the word at index 4 is adapt
dictionary[4] 

In [ ]:
# Checking dictionary created
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

### Gensim filter_extremes

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [ ]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

### Gensim doc2bow

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (it's a pandas series) into the bag-of-words list of tuples (token_id, token_count). Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [ ]:
# Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many words and how many times those words appear. Save this to 'bow_corpus'
bow_corpus = {}
bow_corpus = processed_docs.apply(dictionary.doc2bow)

In [ ]:
# Checking Bag of Words corpus for sample document (document_num=3)
bow_corpus[document_num] # --> Format is (token_id, token_count)

In [ ]:
# Preview BOW for our sample preprocessed document. Here document_num is document number 3 which we have checked in Step 2.
for i,a in bow_corpus[document_num]: # Remember that bow_doc_3 is a list
    print("Word \"{}\" with token_id {} appears {} time.".format(dictionary[i], i, a))

## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [ ]:
# Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [ ]:
# Apply transformation to the entire corpus and call it 'corpus_tfidf'

corpus_tfidf = tfidf[bow_corpus]

In [ ]:
#Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
from pprint import pprint

for i in corpus_tfidf:
    pprint(i)
    break

## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [ ]:
# Train lda model using gensim.models.LdaMulticore and save it to 'lda_model'
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                       num_topics=10,
                                       id2word = dictionary,
                                       passes = 2, # extra parameter for LDA multicore
                                       workers=2 # extra parameter for LDA multicore 
                                      )

In [ ]:
# For each topic, we will explore the words occuring in that topic and its relative weight
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

## Step 4.2 Running LDA using TF-IDF ##

In [ ]:
# Define lda model using corpus_tfidf
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=10, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)

In [ ]:
type(lda_model_tfidf)

In [ ]:
# Exploring the words occuring in each topic and its relative weight
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

### Classification of the topics ###

When using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [ ]:
# Text of sample document 3
len(processed_docs[3])

In [ ]:
# Check which topic our test document belongs to using the LDA Bag of Words model.

# document_num is 3 because our test document is document 3
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

## Step 5.2: Record the top terms of all topics of all documents to a dataframe

In [ ]:
#initialise the array, numer of rows = documents.shape[0] and the number of columns to lda_model_tfidf.get_topics().shape[1].
document_terms = np.zeros((documents.shape[0], lda_model_tfidf.get_topics().shape[1]))

In [ ]:
from tqdm import tqdm

for i in tqdm(range(documents.shape[0])):
    document_terms[i,:] = sum([lda_model_tfidf.get_topics()[k]*v for k,v in lda_model_tfidf.get_document_topics(bow_corpus[i])])
#     break

In [ ]:
column_names = [v for k,v in dictionary.items()]

In [ ]:
for i in range(documents.shape[0]):
    document_terms[i,:] = sum([lda_model_tfidf.get_topics()[k]*v for k,v in lda_model_tfidf.get_document_topics(bow_corpus[i])])

dfterms = pd.DataFrame(document_terms, columns = [v for k,v in dictionary.items()])

In [ ]:
# if wanting to sort and apply threshold to terms
# [dictionary[k] for k in np.argsort(document_terms[0,:])[-10:]]

In [ ]:
dfterms.to_csv('terms.csv', index=False, encoding='utf-8')

## Step 5.3: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [ ]:
from operator import itemgetter, attrgetter

# Check which topic our test document belongs to using the LDA TF-IDF model.
lda_bestprediction = sorted(lda_model_tfidf[bow_corpus[document_num]], key=itemgetter(1), reverse=True)[0][0]
lda_model_tfidf.print_topic(lda_bestprediction, 10)

In [ ]:
# i = 1
# sorted(lda_model_tfidf[bow_corpus[i]], key=itemgetter(1), reverse=True)[0][0]

In [ ]:
# df1=lda_model_tfidf.print_topic(lda_bestprediction, 10)

In [ ]:
# weights = np.asarray(lda_model_tfidf.get_topic_terms(1,10))[:,1]

In [ ]:
# terms = [dictionary[k] for k in np.asarray(lda_model_tfidf.get_topic_terms(1,10))[:,0].astype(int)]

In [ ]:
# pd.DataFrame(lda_model_tfidf.get_topics()).iloc[1].plot()

In [ ]:
# pd.DataFrame(lda_model_tfidf.get_topics()).iloc[0].plot()

In [ ]:
# sum(np.asarray(lda_model_tfidf.get_document_topics(bow_corpus[49]))[:,1])

In [ ]:
# np.sum(np.asarray(lda_model_tfidf.get_topic_terms(0))[:,1])

In [ ]:
# lda_model_tfidf.get_document_topics(bow_corpus[49])   #for document number 49

In [ ]:
# [(lda_model_tfidf.get_topic_terms(k),v) for k,v in lda_model_tfidf.get_document_topics(bow_corpus[49])]

In [ ]:
# lda_model_tfidf.get_topic_terms(4)

In [ ]:
# pd.DataFrame.from_records(list(zip(terms,weights))).head()

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    #print(lda_model_tfidf.print_topic(index, 10))
    print("\nScore: {}\t\nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))   #(topicno, topn=10)

## Step 6: Testing model on unseen document ##

In [ ]:
unseen_document = "Senior financial consultant, Melbourne/Sydney, quantitative modelling risk management."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

The model correctly classifies the unseen document with 'x'% probability to the X category.